In [2]:
import pandas as pd
import numpy as np

# # 10.1 (GroupBy Mechanics)

* INTRODUÇÃO

In [53]:
molduras = pd.read_csv('molduras.csv', skipinitialspace=True).sort_index()

# Estrututa: valor_a_ser_operado.groupby(valores_de_critério).operador_estatístico
molduras['Preço'].groupby(molduras['Cor']).mean()

# Podemos também realizar agregações com múltiplas colunas
molduras['Preço'].groupby([molduras['Cor'],molduras['Madeira']]).mean()

#Lembrete: Com múltiplas colunas a serem agregadas, passá-las dentro de uma lista!

#Size: Um 'count' para os par
a = molduras.groupby(['Madeira','Cor'] ).size()
a.unstack()

# Dica: Utilizar o unstack deixa a estética do resultado do groupby melhor

Cor,Branco,Preto,Vermelho
Madeira,,,
Baobab,1.0,1.0,1.0
Pau-Brasil,NaN,2.0,1.0
Peuplier,1.0,NaN,1.0


* ITERATING OVER GROUPS

In [68]:
temperatura = pd.read_csv('temperatura.csv', skipinitialspace=True)

media_temp = temperatura.groupby('Ano').mean()
display(media_temp)
for name in temperatura.groupby('Ano').mean():
    print(name)

,Temperatura,Umidade
Ano,,
2002,26.0,33.5
2003,24.0,46.5
2004,24.5,36.5


Temperatura
Umidade


In [87]:
# Podemos realizar o groupby em elementos de coluna, por exemplo
a = temperatura.groupby(temperatura.dtypes, axis=1).size()
display(a)

int64     3
object    1
dtype: int64

In [6]:
# Maneira mais precisa de se definir o groupby

temperatura = pd.read_csv('temperatura.csv', skipinitialspace = True)

temperatura.groupby(['Estado'])[['Temperatura']].mean()
#DF.groupby([colunas de index])[[colunas de operações]].operação



,Temperatura
Estado,
RJ,25.000000
SP,24.666667


In [31]:
# Como transoformar colunas distintas em uma única só no groupby

temperatura2 = pd.read_csv('temperatura2.csv', skipinitialspace=True, header=0, index_col=0)

# Vamos sinalizar para o pandas: a Temperatura de 2011 e 2012 será agrupada em Temperatura Média
# O mesmo procedimento é válido para as colunas de umidade
mapping={'Temperatura 2011':'Temperatura Média','Temperatura 2012':'Temperatura Média',
       'Umidade 2011':'Umidade Média','Umidade 2012':'Umidade Média'}

# Estamos indicando: utilize mapping como critério de agregação de dados
temperatura2.groupby([mapping], axis=1).mean()


,Temperatura Média,Umidade Média
Estado,,
SP,22.0,68.5
RJ,24.5,53.5
MG,23.0,64.5
ES,27.0,72.5


* Grouping With Functions

In [63]:
# O método groupby também possui as suas próprias funções de agregação
# Podemos definir a ordem do index com o tamanho dos itens

molduras = pd.read_csv('molduras.csv', skipinitialspace=True, header=0, index_col=0)

molduras.groupby([len,'Cor'])['Preço'].min()

# A renomeação do index não afetará o procedimento
cor_apelido = {'Preto':'Cor 1', 'Vermelho':'Cor 2', 'Branco':'Cor 3'}
molduras.groupby([len, cor_apelido])['Preço'].mean()

5  Cor 1    24.666667
6  Cor 3    27.500000
8  Cor 2    30.000000
Name: Preço, dtype: float64

* Grouping by Index Levels

In [72]:
# Caso os títulos de colunas tenham mais de uma camada, podemos especificá-las também no groupby
temperatura2_novo = pd.read_csv('temperatura2_novo.csv', skipinitialspace=True, header=[0,1], index_col=0)

# Aqui, estamos realizando a média da soma da temperatura e umidade com base em seu ano
temperatura2_novo.groupby(level='Período', axis=1).mean()

#display(temperatura2_novo)

Período,2011,2012
SP,45.0,45.5
RJ,40.0,38.0
MG,39.5,48.0
ES,45.5,54.0


In [86]:
# Outro exemplo do uso de level
# O DF seguinte expressa o número de vitórias de cada time por competição

espanha = pd.read_csv('espanha.csv', skipinitialspace=True, header=[0,1], index_col=0)
espanha.groupby(level=0, axis=1).sum()

Temporada,2019,2020
Barcelona,34,34
Real Madrid,33,36
Atlético Madrid,30,29
Valência,17,18


# #10.2 (Data Aggregation)

In [141]:
# Para o groupby, podemos criar e utilizar as nossas próprias funções de agregação
# Vamos criar uma função que calcula a amplitude dos valores do DF
amp = lambda x: x.max() - x.min()

mapping={'Temperatura 2011':'Temperatura Média','Temperatura 2012':'Temperatura Média',
       'Umidade 2011':'Umidade Média','Umidade 2012':'Umidade Média'}

temperatura = pd.read_csv('temperatura2.csv', skipinitialspace=True, header=0, index_col=0)
temperatura.groupby([mapping],axis=1).mean()
#REVER ISSO

,Temperatura Média,Umidade Média
Estado,,
SP,22.0,68.5
RJ,24.5,53.5
MG,23.0,64.5
ES,27.0,72.5


* Column Wise and Multiple Function Application


In [252]:
# Podemos aplicar múltiplas funções no groupby

boletim = pd.read_csv('boletim.csv', skipinitialspace=True)

# Estamos aqui aplicando a função média e soma e dando um título à coluna de cada função
boletim.groupby('Aluno', axis=0).agg([('Média','mean'), ('Soma','sum')])

# Infelizmente, o .agg não funciona para groupby's com axis=1

P1          P2      
          Média  Soma Média  Soma
Aluno                            
Eduardo    2.40   4.8  6.50  13.0
Felipe     4.95   9.9  6.05  12.1
Guilherme  8.65  17.3  6.35  12.7
Letícia    7.75  15.5  5.00  10.0

In [269]:
# Outra maneira de se aplicar múltiplas funções

# Sem apelidar
functions = ['sum','mean','min']

# Com apelidos
functions_ = [('Soma','sum'), ('Média','mean'), ('Mínimo','min')]

boletim_functions = boletim.groupby('Aluno', axis=0)['P1','P2'].agg(functions_)
boletim_functions

# Podemos printar apenas um fatia desse groupby
display(boletim_functions['P1'])

<ipython-input-269-f5b31641830c>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  boletim_functions = boletim.groupby('Aluno', axis=0)['P1','P2'].agg(functions_)


,Soma,Média,Mínimo
Aluno,,,
Eduardo,4.8,2.40,1.3
Felipe,9.9,4.95,3.4
Guilherme,17.3,8.65,7.8
Letícia,15.5,7.75,6.5


In [276]:
# Podemos escolher aplicar diferentes funções para diferentes colunas

# Para as notas P1, estamos aplicando soma; já para P2, aplicamos a média!
boletim.groupby('Aluno')['P1','P2'].agg({'P1':'sum', 'P2':'mean'})

#Podemos escolher mais de uma função por coluna!
boletim.groupby('Aluno')['P1','P2'].agg({'P1':['sum','max'], 'P2':['mean','min']})

<ipython-input-276-a7b9a478fcc0>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  boletim.groupby('Aluno')['P1','P2'].agg({'P1':'sum', 'P2':'mean'})
<ipython-input-276-a7b9a478fcc0>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  boletim.groupby('Aluno')['P1','P2'].agg({'P1':['sum','max'], 'P2':['mean','min']})


P1         P2     
            sum  max  mean  min
Aluno                          
Eduardo     4.8  3.5  6.50  6.5
Felipe      9.9  6.5  6.05  3.2
Guilherme  17.3  9.5  6.35  3.5
Letícia    15.5  9.0  5.00  3.5

In [278]:
# Apelidando as diferentes funções
boletim.groupby('Aluno')['P1','P2'].agg({'P1':[('Soma','sum')], 'P2':[('Média','mean')]})

<ipython-input-278-a82e52a3bf15>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  boletim.groupby('Aluno')['P1','P2'].agg({'P1':[('Soma','sum')], 'P2':[('Média','mean')]})


,P1,P2
,Soma,Média
Aluno,,
Eduardo,4.8,6.50
Felipe,9.9,6.05
Guilherme,17.3,6.35
Letícia,15.5,5.00


* Returning Aggregated Data Without Row Indexes

In [301]:
# Por questões visuais, podemos retirar o index das tabelas produzidas com groupby

# Para isso, nosso pd.read_csv não deve ter um index definido (não ter index_col!) ??
boletim = pd.read_csv('boletim.csv', skipinitialspace=True)
boletim.groupby(['Aluno'], as_index=False)['P1'].mean()  #.agg(['sum', 'mean'])

# as_index possui uma limitação: ele não funciona caso .agg seja utilizado...


,Aluno,P1
0,Eduardo,2.40
1,Felipe,4.95
2,Guilherme,8.65
3,Letícia,7.75


# #10.3 (General Split-Apply-Combine)

In [26]:
# Explorando o poder do groupby
tips = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / (tips['total_bill'] - tips['tip'])
tips

def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

# O DF produzido aplica a função top para os top 5 casos de smoker = No e smoker = Yes
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.310180
       185       20.69  5.00     No   Sun  Dinner     5  0.318674
       51        10.29  2.60     No   Sun  Dinner     2  0.338101
       149        7.51  2.00     No  Thur   Lunch     2  0.362976
       232       11.61  3.39     No   Sat  Dinner     2  0.412409
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.387973
       183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
       67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
       178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
       172        7.25  5.15    Yes   Sun  Dinner     2  2.452381

In [28]:
# Ainda explorando o poder do groupby
# Podemos especificar, dentro de apply, outros parâmetros da função chamada

tips.groupby('smoker').apply(top, n=1, column='tip')

,,total_bill,tip,smoker,day,time,size,tip_pct
smoker,,,,,,,,
No,212,48.33,9.0,No,Sat,Dinner,4,0.228833
Yes,170,50.81,10.0,Yes,Sat,Dinner,3,0.245038


* Supressing the Group Keys

In [32]:
# Podemos desabilitar o index imposto pelo groupby com group_keys=False

# Dessa forma, o index original do DF será preservado
tips.groupby('smoker', group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.310180
185,20.69,5.00,No,Sun,Dinner,5,0.318674
51,10.29,2.60,No,Sun,Dinner,2,0.338101
149,7.51,2.00,No,Thur,Lunch,2,0.362976
232,11.61,3.39,No,Sat,Dinner,2,0.412409
109,14.31,4.00,Yes,Sat,Dinner,2,0.387973
183,23.17,6.50,Yes,Sun,Dinner,4,0.389922
67,3.07,1.00,Yes,Sat,Dinner,1,0.483092
178,9.60,4.00,Yes,Sun,Dinner,2,0.714286
172,7.25,5.15,Yes,Sun,Dinner,2,2.452381


* Filling Missing Values with Group-Specific Values

In [77]:
# Vamos preencher NA's de acordo com uma determinada regra
boletim = pd.read_csv('boletim_alvaro.csv', skipinitialspace=True)

# Nota-se que há 2 notas perdidas. Uma de Química e uma de Física
display(boletim)

# Vamos criar um lambda para substituir os NA's pela média das provas existentes, por matéria
fill_mean = lambda g: g.fillna(g.mean())

# Com isso, a nota perdida de Química se tornou a média das duas notas existentes de Química
# Idem para Física
boletim.groupby('Matéria', group_keys=False).apply(fill_mean)



,Matéria,Nota
0,Química,4.5
1,Química,NaN
2,Química,6.0
3,Física,3.2
4,Física,NaN
5,Física,9.2


,Matéria,Nota
3,Física,3.20
4,Física,6.20
5,Física,9.20
0,Química,4.50
1,Química,5.25
2,Química,6.00


In [86]:
# Outras maneiras de se agrupar valores como groupby
temperatura = pd.read_csv('estados.csv', skipinitialspace=True)
regiao =  4*['Sudeste'] + 3*['Sul']

# Utilizaremos a lista regiao como parâmetro de categorização dos Estados
# Fizemos o Python identificar quais estados pertencem ao Sudeste, e quais ao Sul
display(temperatura.groupby(regiao).mean()) 

# Agora, vamos fazê-lo preencher os NA's de acordo com a temperatura média de cada região
fill_mean = lambda g: g.fillna(g.mean())

# Note que a temperatura média da Região Sudeste foi destinada a MG, e a do Sul, para PR
temperatura.groupby(regiao).apply(fill_mean)

,Temperatuta
Sudeste,26.666667
Sul,15.000000


Estado  Temperatuta
Sudeste 0     SP    25.000000
        1     MG    26.666667
        2     RJ    27.000000
        3     ES    28.000000
Sul     4     RS    13.000000
        5     SC    17.000000
        6     PR    15.000000

In [5]:
# Por outro lado, podemos, também, preencher os NA's com valores arbitrários
boletim = pd.read_csv('boletim_alvaro.csv', skipinitialspace=True)

# Criamos um dicionário com as notas restantes e suas respectivas matérias
notas_restantes= {'Química':9.5, 'Física':6.8}

# Em seguida, constuímos uma função capaz de preencher os NA's conforme o nome do groupby
preencher_notas = lambda g: g.fillna(notas_restantes[g.name])
boletim.groupby('Matéria').apply(preencher_notas)

# CONTINUAR DAQUI! TIVE QUE PARAR NO MEIO!

,Matéria,Nota
0,Química,4.5
1,Química,9.5
2,Química,6.0
3,Física,3.2
4,Física,6.8
5,Física,9.2


* Random Sampling and Permutation

In [16]:
# Vamos montar um baralho de cartas

suits = ['H','S','C', 'D']
card_val = (list(range(1,11)) +[10]*3) *4
base_names = ['A'] +list(range(2,11)) + ['J','K','Q']
cards= []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [18]:
# Para criar a nossa mão com cartas aleatórias, poderíamos criar utlizar o comando .sample()

# O lambda retornará um sample de deck contendo 5 de seus elementos
samp = lambda g,n: g.sample(n)

samp(deck,5)

4D     4
7H     7
3D     3
JS    10
3C     3
dtype: int64

In [13]:
# Partindo para algo mais complexo: vamos pegar dois cards aleatórios por suit

get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(samp, n=2)

C  5C     5
   KC    10
D  6D     6
   KD    10
H  AH     1
   4H     4
S  3S     3
   KS    10
dtype: int64

* Group Weighted Average and Correlaton

In [46]:
# Vamos calcular a média ponderada das notas de um boletim
ponderada = pd.read_csv('ponderada.csv')
display(ponderada)

media_p = lambda g: np.average(g.iloc[:,1], weights = g.iloc[:,2])
ponderada.groupby('Disciplina').apply(media_p)

,Disciplina,Notas,Peso
0,Química,4.3,3
1,Química,6.5,3
2,Química,5.1,2
3,Química,8.0,2
4,Literatura,10.0,3
5,Literatura,3.0,3
6,Literatura,6.5,2
7,Literatura,2.5,2


Disciplina
Literatura    5.70
Química       5.86
dtype: float64

In [8]:
close_px = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px.csv',
                      parse_dates=True, index_col=0)

spx_corr = lambda x: x.corrwith(x.iloc[:,7])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078
1995,0.291532,0.161829,0.519126,0.416390,0.315733,0.453660,0.413144,1.0,0.368752
1996,0.292344,0.191482,0.750724,0.388497,0.569232,0.564015,0.421477,1.0,0.538736
1997,0.564427,0.211435,0.827512,0.646823,0.703538,0.606171,0.509344,1.0,0.695653
1998,0.533802,0.379883,0.815243,0.623982,0.591988,0.698773,0.494213,1.0,0.369264
1999,0.099033,0.425584,0.710928,0.486167,0.517061,0.631315,0.336593,1.0,0.315383


In [10]:
# Poderíamos também aplicar uma correlação entre apenas duas ações

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])

SyntaxError: unexpected EOF while parsing (<ipython-input-10-024af2e2dde6>, line 3)

* Group-Wise Linear Regression

In [11]:
import statsmodels.api as sm
def regress (data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])



,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
1995,0.858818,-0.001423
1996,0.829389,-0.001791
1997,0.749928,-0.001901
1998,1.164582,0.004075
1999,1.384989,0.003273


# #10.4 (Pivot Tables and Cross-Tabulation)

In [3]:
# Vamos agora criar pivot tables com o Pandas
tips = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / (tips['total_bill'] - tips['tip'])

# Por padrão, pivot tables têm mean() como função de agregação
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.179740   18.420000
     Yes     2.066667  2.714000  0.216293   16.813333
Sat  No      2.555556  3.102889  0.190412   19.661778
     Yes     2.476190  2.875476  0.179833   21.276667
Sun  No      2.929825  3.167895  0.193617   20.506667
     Yes     2.578947  3.516842  0.322021   24.120000
Thur No      2.488889  2.673778  0.193424   17.113111
     Yes     2.352941  3.030000  0.198508   19.190588

In [4]:
# No primeiro instante, o comando parece ser idêntico ao groupby, mas podemos fazer operações mais complexas...
# Podemos, com pivot_table, escolher quais colunas de nosso DF serão disponibilizadas
# Margins será equivalente à coluna Total das pivot tables do Excel

tips.pivot_table(['tip_pct','size'], index=['time','day'], columns=['smoker'], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.162612  0.202545  0.192562
       Sat   2.555556  2.476190  2.517241  0.190412  0.179833  0.185305
       Sun   2.929825  2.578947  2.842105  0.193617  0.322021  0.225718
       Thur  2.000000       NaN  2.000000  0.190114       NaN  0.190114
Lunch  Fri   3.000000  1.833333  2.000000  0.231125  0.236915  0.236088
       Thur  2.500000  2.352941  2.459016  0.193499  0.198508  0.194895
All          2.668874  2.408602  2.569672  0.192237  0.218176  0.202123

In [11]:
# Podemos escolher uma outra função de agregação a ser utilizada
# Para preencher NA's, utilizar fill_value
tips.pivot_table('size', index=['time', 'day'], columns=['smoker'], aggfunc=['count'], fill_value=0, margins=True, margins_name='Oi').rename(columns={'count':'Contador'})

#  Temos maior liberdade de estruturação da tabela com pivot_table

Contador         
smoker            No Yes   Oi
time   day                   
Dinner Fri         3   9   12
       Sat        45  42   87
       Sun        57  19   76
       Thur        1   0    1
Lunch  Fri         1   6    7
       Thur       44  17   61
Oi               151  93  244